# **User 관심도 점수를 카드 혜택에 맞춰 카드별 매칭 점수 도출**
# 1. 필요한 라이브러리 및 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
card = pd.read_csv('/content/drive/MyDrive/CARD-LANGCHAIN/data/card_data.csv')
interests = pd.read_csv('/content/drive/MyDrive/CARD-LANGCHAIN/data/관심도모델링1차데이터.csv')

# 2. 카드 혜택 대분류 카테고리 뽑기

In [3]:
card.head()

,Card Name,Annual_fee,Base Record,Category_List
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,['항공마일리지']
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"['쇼핑', '레저', '카페/베이커리', '편의점', '대형마트', '의료', '..."
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"['쇼핑', '주유', '카페/베이커리', '편의점', '대중교통', '간편결제',..."
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"['대중교통', '간편결제', '편의점', '통신', '카페/베이커리', '대형마트..."
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"['포인트/캐시백', '카페/베이커리', '쇼핑', '통신', '대중교통', '영화..."


In [4]:
import ast
card['Category_List'] = card['Category_List'].apply(ast.literal_eval)

In [5]:
category_dict = {
'항공' : '항공마일리지',
'쇼핑/간편결제' : ['쇼핑', '간편결제' , '포인트/캐시백']
'편의점/대형마트' : ['편의점', '대형마트'],
'카페/베이커리' :['카페', '베이커리'],
'외식' : '외식',
'의료' : '의료'
,'반려동물': '반려동물'
,'뷰티': '뷰티'
,'교통/주유' : ['대중교통', '주유', '하이패스'],
'교육/육아': ['교육', '육아'],
'문화': ['문화', '레저', '영화'],
'납부': ['통신', '관리비', '납부 혜택']}

# 매핑 함수
def map_category_list_to_key(category_list):
  keys = set()
  for category in category_list:
    for key, values in category_dict.items():
        if category in values:
            keys.add(key)
  return list(keys)

# 대분류 카테고리 컬럼 추가
card['category'] = card['Category_List'].apply(map_category_list_to_key)

card.head()

,Card Name,Annual_fee,Base Record,Category_List,category
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,[항공마일리지],[항공]
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"[쇼핑, 레저, 카페/베이커리, 편의점, 대형마트, 의료, 뷰티, 대중교통, 외식]","[쇼핑/간편결제, 의료, 뷰티, 교통/주유, 문화, 카페/베이커리]"
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"[쇼핑, 주유, 카페/베이커리, 편의점, 대중교통, 간편결제, 대형마트, 외식, 통...","[쇼핑/간편결제, 의료, 반려동물, 항공, 뷰티, 납부, 교통/주유, 문화, 교육/..."
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"[대중교통, 간편결제, 편의점, 통신, 카페/베이커리, 대형마트, 주유, 외식, 쇼...","[쇼핑/간편결제, 의료, 반려동물, 뷰티, 납부, 교통/주유, 문화, 교육/육아, ..."
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"[포인트/캐시백, 카페/베이커리, 쇼핑, 통신, 대중교통, 영화, 편의점]","[문화, 납부, 교통/주유, 쇼핑/간편결제]"


# 3. 혜택 별 카드 매칭

In [6]:
card['category']

,category
0,[항공]
1,"[쇼핑/간편결제, 의료, 뷰티, 교통/주유, 문화, 카페/베이커리]"
2,"[쇼핑/간편결제, 의료, 반려동물, 항공, 뷰티, 납부, 교통/주유, 문화, 교육/..."
3,"[쇼핑/간편결제, 의료, 반려동물, 뷰티, 납부, 교통/주유, 문화, 교육/육아, ..."
4,"[문화, 납부, 교통/주유, 쇼핑/간편결제]"
...,...
161,"[쇼핑/간편결제, 의료, 뷰티, 납부, 교통/주유, 문화, 교육/육아, 카페/베이커리]"
162,[항공]
163,"[문화, 카페/베이커리, 쇼핑/간편결제]"
164,"[교통/주유, 쇼핑/간편결제]"


In [7]:
interests.head()

,user_id,category,암묵적관심도,명시적관심도
0,1,교육/육아,1,0
1,1,문화,0,1
2,1,납부,0,1
3,1,항공,0,1
4,1,반려동물,1,0


user_id별로 관심 카테고리가 있는 카드 혜택을 매칭시켜
암묵적 관심도는 *0.7, 명시적관심도 * 0.9 를 곱해
매칭 점수가 가장 큰 카드뽑기

In [8]:
def map_category_list_to_key(category_list):
  keys = set()
  for category in category_list:
    for key, values in category_dict.items():
        if category in values:
            keys.add(key)
  return list(keys)

In [9]:
card

,Card Name,Annual_fee,Base Record,Category_List,category
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,[항공마일리지],[항공]
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"[쇼핑, 레저, 카페/베이커리, 편의점, 대형마트, 의료, 뷰티, 대중교통, 외식]","[쇼핑/간편결제, 의료, 뷰티, 교통/주유, 문화, 카페/베이커리]"
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"[쇼핑, 주유, 카페/베이커리, 편의점, 대중교통, 간편결제, 대형마트, 외식, 통...","[쇼핑/간편결제, 의료, 반려동물, 항공, 뷰티, 납부, 교통/주유, 문화, 교육/..."
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"[대중교통, 간편결제, 편의점, 통신, 카페/베이커리, 대형마트, 주유, 외식, 쇼...","[쇼핑/간편결제, 의료, 반려동물, 뷰티, 납부, 교통/주유, 문화, 교육/육아, ..."
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"[포인트/캐시백, 카페/베이커리, 쇼핑, 통신, 대중교통, 영화, 편의점]","[문화, 납부, 교통/주유, 쇼핑/간편결제]"
...,...,...,...,...,...
161,LOCA PLATINUM(할인형),"국내 10만원, 해외 10만원, 가족겸용 없음",조건없음,"[카페/베이커리, 쇼핑, 뷰티, 대형마트, 편의점, 주유, 외식, 영화, 통신, 교...","[쇼핑/간편결제, 의료, 뷰티, 납부, 교통/주유, 문화, 교육/육아, 카페/베이커리]"
162,I-MILEAGE(아시아나),"국내 4만1천원, 해외 4만2천원",직전 1개월 합계 50만원 이상,"[항공마일리지, 카페/베이커리]",[항공]
163,The CJ-현대카드M Edition2,"국내 1만5천원, 해외 2만원",조건없음,"[쇼핑, 영화, 외식, 카페/베이커리, 포인트/캐시백]","[문화, 카페/베이커리, 쇼핑/간편결제]"
164,KB국민 그린카드(서울형),"국내전용 없음, 해외겸용 없음",직전 1개월 합계 20만원 이상,"[포인트/캐시백, 대중교통, 쇼핑, 대형마트]","[교통/주유, 쇼핑/간편결제]"


In [10]:
card[
    'category'
][3][1]

'의료'

In [15]:
interests.head(10)

,user_id,category,암묵적관심도,명시적관심도
0,1,교육/육아,1,0
1,1,문화,0,1
2,1,납부,0,1
3,1,항공,0,1
4,1,반려동물,1,0
5,1,쇼핑/간편결제,1,0
6,2,교육/육아,1,0
7,2,문화,0,1
8,2,뷰티,0,1
9,2,반려동물,1,0


In [12]:
card.head()

,Card Name,Annual_fee,Base Record,Category_List,category
0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,[항공마일리지],[항공]
1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"[쇼핑, 레저, 카페/베이커리, 편의점, 대형마트, 의료, 뷰티, 대중교통, 외식]","[쇼핑/간편결제, 의료, 뷰티, 교통/주유, 문화, 카페/베이커리]"
2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"[쇼핑, 주유, 카페/베이커리, 편의점, 대중교통, 간편결제, 대형마트, 외식, 통...","[쇼핑/간편결제, 의료, 반려동물, 항공, 뷰티, 납부, 교통/주유, 문화, 교육/..."
3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"[대중교통, 간편결제, 편의점, 통신, 카페/베이커리, 대형마트, 주유, 외식, 쇼...","[쇼핑/간편결제, 의료, 반려동물, 뷰티, 납부, 교통/주유, 문화, 교육/육아, ..."
4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"[포인트/캐시백, 카페/베이커리, 쇼핑, 통신, 대중교통, 영화, 편의점]","[문화, 납부, 교통/주유, 쇼핑/간편결제]"


In [27]:
interests.groupby('user_id')['category'].count().reset_index()

,user_id,category
0,1,6
1,2,5
2,3,4
3,4,4
4,5,3
5,6,5
6,7,3
7,8,7
8,9,5
9,10,7


In [28]:
count = interests.groupby('user_id')['category'].count().reset_index()
count

,user_id,category
0,1,6
1,2,5
2,3,4
3,4,4
4,5,3
5,6,5
6,7,3
7,8,7
8,9,5
9,10,7


In [30]:
count[count['user_id'] == 1]['category']

,category
0,6


In [32]:

matching_card= []

count = interests.groupby('user_id')['category'].count().reset_index()

for i, card_row in card.iterrows():
  total_score = 0
  for idx, user_row in interests.iterrows():
    score = 0
    # 카드의 혜택 카테고리와 사용자의 관심 카테고리 비교
    if user_row['category'] in card_row['category']:

      # 암묵적 관심도 점수
      if user_row['암묵적관심도'] >= 1:
        user_count = count[count['user_id'] == user_row['user_id']]['category'].values[0]
        score += user_row['암묵적관심도'] / user_count
      if user_row['명시적관심도'] == 1:
        score += user_row['명시적관심도']

        # 매칭 데이터 기록
    matching_card.append({
            'user_id': user_row['user_id'],
            'category': user_row['category'],
            'Card_category': card_row['category'],
            'Card Name': card_row['Card Name'],
            'matching_score': score
        })

matching_card_df = pd.DataFrame(matching_card)

# 카드별 총 점수 계산
final_scores = matching_card_df.groupby(['user_id','Card Name']).agg({
    'matching_score': 'sum'
}).reset_index()

final_scores

,user_id,Card Name,matching_score
0,1,ALL 우리카드 Infinite,0.166667
1,1,ALL 우리카드 Premium,1.166667
2,1,BC 바로 K-패스 카드,2.166667
3,1,BC 바로 MACAO 카드,0.166667
4,1,BC 바로 리워드 플러스,2.333333
...,...,...,...
1655,10,현대카드Z play,1.142857
1656,10,현대카드Z work Edition2,1.285714
1657,10,현대카드ZERO Edition3(포인트형),2.571429
1658,10,현대카드ZERO Edition3(할인형),2.571429


In [33]:
matching_card_df


,user_id,category,Card_category,Card Name,matching_score
0,1,교육/육아,[항공],하나 트래블로그 SKYPASS 신용카드,0.000000
1,1,문화,[항공],하나 트래블로그 SKYPASS 신용카드,0.000000
2,1,납부,[항공],하나 트래블로그 SKYPASS 신용카드,0.000000
3,1,항공,[항공],하나 트래블로그 SKYPASS 신용카드,1.000000
4,1,반려동물,[항공],하나 트래블로그 SKYPASS 신용카드,0.000000
...,...,...,...,...,...
8129,10,반려동물,"[문화, 교통/주유, 쇼핑/간편결제]",KB국민 Green Wave 1.5℃ 카드,0.000000
8130,10,항공,"[문화, 교통/주유, 쇼핑/간편결제]",KB국민 Green Wave 1.5℃ 카드,0.000000
8131,10,교육/육아,"[문화, 교통/주유, 쇼핑/간편결제]",KB국민 Green Wave 1.5℃ 카드,0.000000
8132,10,쇼핑/간편결제,"[문화, 교통/주유, 쇼핑/간편결제]",KB국민 Green Wave 1.5℃ 카드,0.142857


In [34]:
final_scores[final_scores['user_id'] == 6].sort_values('matching_score')

,user_id,Card Name,matching_score
838,6,CLUB Signature SKYPASS,0.0
845,6,I-MILEAGE(아시아나),0.0
844,6,I-MILEAGE(대한항공),0.0
951,6,싱가포르항공 크리스플라이어 더 베스트 신한카드,0.0
969,6,트래블로그 PRESTIGE 신용카드,0.0
...,...,...,...
965,6,카드의정석 EVERY POINT,4.0
863,6,LOCA 100,4.0
865,6,LOCA CLASSIC (로카 클래식),4.0
963,6,카드의정석 EVERY DISCOUNT,4.0


In [41]:
final_scores[(final_scores['user_id'] == 6) & (final_scores['Card Name'] == 'KB국민 와이즈카드')]

,user_id,Card Name,matching_score
858,6,KB국민 와이즈카드,4.0


In [39]:
card[card['Card Name']=='KB국민 와이즈카드']['category']

,category
46,"[쇼핑/간편결제, 의료, 뷰티, 납부, 교통/주유, 문화, 교육/육아, 카페/베이커리]"


In [40]:
interests[interests['user_id']==6]

,user_id,category,암묵적관심도,명시적관심도
22,6,뷰티,1,1
23,6,쇼핑/간편결제,2,0
24,6,교통/주유,1,1
25,6,교육/육아,1,0
26,6,납부,0,1


In [ ]:
pd.set_option('display.max_colwidth', None)

# 4. 연회비

In [42]:
# 국내 금액 추출 함수
def extract_domestic_fee(fee_string):
    import re
    # 국내 금액만 추출
    match = re.search(r'국내 (?:(\d+)만)?(\d*)천?원?', fee_string)
    if match:
        # '만'과 '천' 단위 합산하여 숫자로 변환
        ten_thousand = int(match.group(1)) * 10000 if match.group(1) else 0
        thousand = int(match.group(2)) * 1000 if match.group(2) else 0
        return ten_thousand + thousand
    return 0  # 국내 금액이 없으면 0 반환

# 데이터프레임의 Annual_fee 열에서 국내 금액 추출
card['Domestic_Fee'] = card['Annual_fee'].apply(extract_domestic_fee)


# 5. 카드 혜택 매칭 점수와 연회비 함께 본 후, 최종 카드 선정

In [43]:
final_scores

,user_id,Card Name,matching_score
0,1,ALL 우리카드 Infinite,0.166667
1,1,ALL 우리카드 Premium,1.166667
2,1,BC 바로 K-패스 카드,2.166667
3,1,BC 바로 MACAO 카드,0.166667
4,1,BC 바로 리워드 플러스,2.333333
...,...,...,...
1655,10,현대카드Z play,1.142857
1656,10,현대카드Z work Edition2,1.285714
1657,10,현대카드ZERO Edition3(포인트형),2.571429
1658,10,현대카드ZERO Edition3(할인형),2.571429


In [44]:
data = pd.merge(card, final_scores, how='right', on='Card Name')

In [45]:
sorted_data = data.groupby(['user_id','Card Name'])[['matching_score','Domestic_Fee']].sum().sort_values(by=['user_id','matching_score','Domestic_Fee'],ascending=[True,False,False])

In [46]:
sorted_data

matching_score  Domestic_Fee
user_id Card Name                                                   
1       삼성카드 & MILEAGE PLATINUM(스카이패스)        3.500000         47000
        the Pink Edition2                     2.500000        150000
        현대카드MM                                2.500000         70000
        현대카드X                                 2.500000         50000
        현대카드M                                 2.500000         30000
...                                                ...           ...
10      현대카드X HYBRID                          0.142857          2000
        현대백화점 FIT카드                           0.142857             0
        현대백화점 해피포인트카드                         0.142857             0
        현대백화점 현대오일뱅크카드                        0.142857             0
        현대백화점카드                               0.142857             0

[1660 rows x 2 columns]

In [48]:
sorted_data.reset_index().sort_values(['user_id','matching_score','Domestic_Fee'],ascending=[True, False, True]).groupby('user_id').head(1)

,user_id,Card Name,matching_score,Domestic_Fee
0,1,삼성카드 & MILEAGE PLATINUM(스카이패스),3.500000,47000
174,2,삼성 iD SIMPLE 카드,3.600000,7000
340,3,삼성 iD SIMPLE 카드,4.250000,7000
506,4,삼성 iD SIMPLE 카드,3.500000,7000
692,5,롯데 국민행복카드,3.333333,0
852,6,롯데 국민행복카드,4.000000,0
1042,7,롯데 국민행복카드,3.000000,0
1162,8,삼성카드 & MILEAGE PLATINUM(스카이패스),3.571429,47000
1328,9,삼성카드 & MILEAGE PLATINUM(스카이패스),3.400000,47000
1494,10,삼성카드 & MILEAGE PLATINUM(스카이패스),3.571429,47000


In [ ]:
card['Annual_fee'].loc[card['Card Name'] == '삼성카드 & MILEAGE PLATINUM(스카이패스)']

,Annual_fee
2,"국내 4만7천원, 해외 4만9천원"
